In [10]:
import yaml

# Check if study object exists, otherwise create a placeholder or load from existing params
if 'study' in locals():
    best_params = study.best_params
elif 'params' in locals():
    best_params = params
else:
    print("Warning: Neither 'study' nor 'params' found. Please run the Optuna optimization first.")
    best_params = {}

# Save to YAML file
file_name = "optuna_best_params_xgboost_hourly.yaml"
with open(file_name, "w") as f:
    yaml.dump(best_params, f, default_flow_style=False, sort_keys=False)

print("✔ Đã lưu:", file_name)

✔ Đã lưu: optuna_best_params_xgboost_hourly.yaml


In [ ]:
import os
import joblib
import skl2onnx
from skl2onnx.common.data_types import FloatTensorType
import numpy as np

# 1️⃣ Tìm mô hình từ thư mục hiện tại hoặc thư mục models
model_path = None
model_filename = None

# Tìm file .pkl hoặc .joblib trong thư mục hiện tại và thư mục cha
search_dirs = [
    ".",
    "..",
    "../models",
    "../daily/models"
]

for search_dir in search_dirs:
    if os.path.exists(search_dir):
        for file in os.listdir(search_dir):
            if file.endswith(('.pkl', '.joblib')) and 'model' in file.lower():
                model_path = os.path.join(search_dir, file)
                model_filename = file
                break
    if model_path:
        break

if model_path and os.path.exists(model_path):
    print(f"✅ Tìm thấy mô hình: {model_path}")
    
    # 2️⃣ Load mô hình
    model = joblib.load(model_path)
    print(f"✅ Đã tải mô hình: {model}")
    
    # 3️⃣ Chuyển đổi sang ONNX
    try:
        # Định nghĩa input type (giả sử mô hình nhận vector đặc trưng)
        initial_type = [('float_input', FloatTensorType([None, 10]))]  # Thay 10 bằng số đặc trưng thực tế
        
        # Chuyển đổi
        onnx_model = skl2onnx.convert_sklearn(model, initial_types=initial_type)
        
        # Lưu file ONNX
        output_path = model_filename.replace('.pkl', '.onnx').replace('.joblib', '.onnx')
        with open(output_path, 'wb') as f:
            f.write(onnx_model.SerializeToString())
        
        print(f"✅ Mô hình đã được export sang ONNX: {output_path}")
    except Exception as e:
        print(f"⚠️ Lỗi khi chuyển đổi sang ONNX: {e}")
        print("💡 Gợi ý: Nếu mô hình là LightGBM, XGBoost, hoặc CatBoost, hãy sử dụng phương pháp chuyên biệt cho từng thư viện")
else:
    print("❌ Không tìm thấy file mô hình (.pkl hoặc .joblib) trong các thư mục tìm kiếm")
    print(f"📁 Các thư mục tìm kiếm: {search_dirs}")

In [ ]:
# ==================== XUẤT MÔ HÌNH ONNX CHO XGBoost/LightGBM ====================
import os
import joblib

# Phương pháp 1: Dùng skl2onnx (cho các mô hình scikit-learn)
try:
    import skl2onnx
    from skl2onnx.common.data_types import FloatTensorType
    
    # Tìm mô hình XGBoost hoặc LightGBM
    model_path = None
    for search_dir in [".", "..", "../models", "../daily/models"]:
        if os.path.exists(search_dir):
            for file in os.listdir(search_dir):
                if ("xgboost" in file.lower() or "lgb" in file.lower() or "lightgbm" in file.lower()) and file.endswith(('.pkl', '.joblib')):
                    model_path = os.path.join(search_dir, file)
                    break
        if model_path:
            break
    
    if model_path:
        model = joblib.load(model_path)
        print(f"✅ Tải mô hình: {model_path}")
        
        # Số features cần phù hợp với mô hình của bạn
        initial_type = [('float_input', FloatTensorType([None, 10]))]
        onnx_model = skl2onnx.convert_sklearn(model, initial_types=initial_type)
        
        output_path = model_path.replace('.pkl', '.onnx').replace('.joblib', '.onnx')
        with open(output_path, 'wb') as f:
            f.write(onnx_model.SerializeToString())
        
        print(f"✅ Xuất ONNX thành công: {output_path}")
    else:
        print("❌ Không tìm thấy mô hình XGBoost/LightGBM")
        
except Exception as e:
    print(f"⚠️ Phương pháp skl2onnx thất bại: {e}")

# Phương pháp 2: Dùng onnx-converter-common cho XGBoost
print("\n--- Phương pháp 2: Dùng onnxmltools ---")
try:
    import onnxmltools
    import xgboost as xgb
    
    # Tìm mô hình XGBoost
    model_path = None
    for search_dir in [".", "..", "../models", "../daily/models"]:
        if os.path.exists(search_dir):
            for file in os.listdir(search_dir):
                if "xgboost" in file.lower() and file.endswith(('.pkl', '.joblib', '.json')):
                    model_path = os.path.join(search_dir, file)
                    break
        if model_path:
            break
    
    if model_path:
        model = joblib.load(model_path)
        print(f"✅ Tải mô hình XGBoost: {model_path}")
        
        # Chuyển đổi XGBoost sang ONNX
        onnx_model = onnxmltools.convert_xgboost(model)
        
        output_path = model_path.replace('.pkl', '.onnx').replace('.joblib', '.onnx')
        onnxmltools.utils.save_model(onnx_model, output_path)
        
        print(f"✅ Xuất ONNX thành công: {output_path}")
    else:
        print("ℹ️ Không tìm thấy mô hình XGBoost")
        
except Exception as e:
    print(f"⚠️ Phương pháp onnxmltools thất bại: {e}")

# Phương pháp 3: Dùng skl2onnx cho LightGBM
print("\n--- Phương pháp 3: LightGBM với skl2onnx ---")
try:
    import lightgbm as lgb
    from skl2onnx.common.data_types import FloatTensorType
    import skl2onnx
    
    # Tìm mô hình LightGBM
    model_path = None
    for search_dir in [".", "..", "../models", "../daily/models"]:
        if os.path.exists(search_dir):
            for file in os.listdir(search_dir):
                if ("lgb" in file.lower() or "lightgbm" in file.lower()) and file.endswith(('.pkl', '.joblib')):
                    model_path = os.path.join(search_dir, file)
                    break
        if model_path:
            break
    
    if model_path:
        model = joblib.load(model_path)
        print(f"✅ Tải mô hình LightGBM: {model_path}")
        
        # Chuyển đổi LightGBM sang ONNX
        initial_type = [('float_input', FloatTensorType([None, 10]))]
        onnx_model = skl2onnx.convert_sklearn(model, initial_types=initial_type)
        
        output_path = model_path.replace('.pkl', '.onnx').replace('.joblib', '.onnx')
        with open(output_path, 'wb') as f:
            f.write(onnx_model.SerializeToString())
        
        print(f"✅ Xuất ONNX thành công: {output_path}")
    else:
        print("ℹ️ Không tìm thấy mô hình LightGBM")
        
except Exception as e:
    print(f"⚠️ Phương pháp LightGBM thất bại: {e}")

print("\n" + "="*50)
print("✅ Hoàn thành xuất mô hình sang ONNX!")